# Vocab Analysis 
## Section 2: Prepare the Data

### 1. Import libraries

In [708]:
import pandas as pd
import sqlite3
import json
from datetime import datetime, timedelta, date
import time

### 2. Import Data

In [709]:
location = "datasets/collection.anki2"
cnx = sqlite3.connect(location) # create sql file connection

In [710]:
# TDD backbone assertion to confirm a function call returns the desired result
def assertEquals(actual, expected, desc):
    assert(actual==expected), desc + " result: " + str(actual) + ", expected: " + str(expected)
    return "OK"

### A word anout how the data was stored

Anki, the Open Source, Spaced Repetition software (& app & service), saves a student's data in a few locations. There are the "Notes" which are the raw info used to make cards (fields of vocab data, metatags, trivia facts, images, audio, etc..) Then there are the "Cards" which are the actual studied items, where the study overview data is stored (such as study date-times, repetitions (reviews), intervals (how long a card is to be remembered), lapses (forgets & subsequent interval resets)). Additionally, data concerning the entire collection is stored under something cryptically called "Columns". Lastly, there is a "RevLog" which contains all the study data in detail for each individual repetition (study datetime, card studied, etc..) This document was critical to piecing together the puzzle: https://github.com/ankidroid/Anki-Android/wiki/Database-Structure

### 3. Extract Deck Creation Date

In [711]:
df_c = pd.read_sql_query("SELECT * FROM col", cnx)
crt = df_c['crt'][0] # save collection creation date (in epoch time)
pd_crt = pd.to_datetime(crt, unit = 's')
print(pd_crt)

assertEquals(str(pd_crt), "2013-01-08 09:00:00", "Collection Creation Date")

2013-01-08 09:00:00


'OK'

### 4. Extract field names to label columns

In [712]:
field_names = []
for row_index, blob in df_c['models'].items():
    for model_id, data in json.loads(blob).items():
        field_names += list(map(lambda fld: fld['name'], data['flds']))
field_names.append('Tags')
expected_names = ['Term', 'Yomi1', 'Translation', 'Translation2', 'Translation3', 'AlternateForms', 'PartOfSpeech', 'Sound', 'Sound2', 'Sound3', 'Examples', 'ExamplesAudio', 'AtoQ', 'AtoQaudio', 'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'Meaning1', 'SimilarWords', 'RelatedWords', 'Breakdown1', 'Comparison', 'Usage', 'Prompt1', 'Prompt2', 'KakuMCD', 'IuMCD', 'ExtraMemo', 'Yomi2', 'Meaning2', 'Breakdown2', 'Picture1', 'Picture2', 'Picture3', 'Picture4', 'HinshiMarker', 'Hint', 'Term2', 'ArabicNumeral', 'CounterKanji', 'Mnemonic', 'SameSoundWords', 'Yomi3', 'gChap', 'gBook', 'semester', 'gNumber', 'Transliteration', 'SoloLookCards', 'TagOverflow', 'blank1', 'blank2', 'Tags']

In [713]:
assertEquals(field_names, expected_names, "Field Names")

'OK'

### 5. Import card data into data frame "df_cards"

In [714]:
# Step 6: Take in study data from Anki collection
df_cards = pd.read_sql_query("SELECT * FROM cards", cnx)
assertEquals(df_cards.shape[0],19314,"Rows")#6386, 21979, 19363
assertEquals(df_cards.shape[1],18,"Columns")

'OK'

### 6. Confirm that card data model matches expected format

In [715]:
expected_columns_1 = ['id', 'nid', 'did', 'ord', 'mod', 'usn', 'type', 'queue', 'due', 'ivl', 'factor',
 'reps', 'lapses', 'left', 'odue', 'odid', 'flags', 'data']

def lists_equal(a,b):
    return (a == b).all()

assertEquals(lists_equal(df_cards.columns.values, expected_columns_1), True, "Card Columns Import")

'OK'

### 7. Shallow check for duplicates (matching rows)

In [716]:
 def has_dupes(df_in):
    dupe = df_in.duplicated()
    return df_in.loc[dupe].shape[0] != 0

In [717]:
assertEquals(has_dupes(df_cards), False, "Duplicates Not Found")

'OK'

### 8. Remove unneeded card dataframe columns

In [718]:
def print_line_break():
    print("-"*75)

In [719]:
def print_before_after(b, a, t=""):
    if t != "":
        print_line_break()
        print(t)
    print_line_break()
    print("Before: " + str(b))
    print_line_break()
    print("After: " + str(a))
    print_line_break()

In [720]:
df_cards_001_less_cols = df_cards.copy()
df_cards_001_less_cols = df_cards_001_less_cols.drop(['did','usn','type','mod','left','odue','odid','flags','data'],axis=1)
expected_columns_2 = ['id', 'nid', 'ord', 'queue', 'due', 'ivl', 'factor', 'reps','lapses']

print_before_after(df_cards.columns.values, df_cards_001_less_cols.columns.values,"Card Columns:")

assertEquals(lists_equal(df_cards_001_less_cols.columns.values, expected_columns_2), True, "Card Model Slimmed")

---------------------------------------------------------------------------
Card Columns:
---------------------------------------------------------------------------
Before: ['id' 'nid' 'did' 'ord' 'mod' 'usn' 'type' 'queue' 'due' 'ivl' 'factor'
 'reps' 'lapses' 'left' 'odue' 'odid' 'flags' 'data']
---------------------------------------------------------------------------
After: ['id' 'nid' 'ord' 'queue' 'due' 'ivl' 'factor' 'reps' 'lapses']
---------------------------------------------------------------------------


'OK'

### 9. Import notes (terms/words) into data frame "df_notes"

In [721]:
# let's take in the 'notes' table, and explicitly save the note id ("nid") 
df_notes = pd.read_sql_query("SELECT * FROM notes", cnx)
df_notes = df_notes.rename(columns={'id':'nid'})

In [722]:
assertEquals(df_notes.shape[0],8381,"Rows") # 2791, 9784, 8403
assertEquals(df_notes.shape[1],11,"Columns")

'OK'

### 10. Remove (drop) unneeded fields (columns)

In [723]:
df_notes_old_col_vals = df_notes.columns.values
df_notes = df_notes.drop(['guid','mid','usn','sfld','csum','flags','data'],axis=1)
#print(df_notes.columns.values)
print_before_after(df_notes_old_col_vals, df_notes.columns.values)

---------------------------------------------------------------------------
Before: ['nid' 'guid' 'mid' 'mod' 'usn' 'tags' 'flds' 'sfld' 'csum' 'flags' 'data']
---------------------------------------------------------------------------
After: ['nid' 'mod' 'tags' 'flds']
---------------------------------------------------------------------------


### 11. Split "fields" column into multiple, assign field names, drop combined col

In [724]:
def time_it(func, *args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    # https://stackoverflow.com/questions/8885663/how-to-format-a-floating-number-to-fixed-width-in-python
    print("{:.0f}".format((end - start)*1000) + " miliseconds")

In [725]:
for i in range(0,len(expected_names)-1):
    df_notes[expected_names[i]] = df_notes.flds.str.split('').str.get(i)
assertEquals('flds' in df_notes.columns.values, True, "'flds' Column Found")
df_notes = df_notes.drop(['flds'],axis=1)
assertEquals('flds' not in df_notes.columns.values, True, "'flds' Column Not Found")
print(df_notes.columns.values)

['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2']


In [726]:
# make sure that all HTML tags have been removed from every card
assertEquals(df_notes[df_notes['Term'].str.contains("</div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Term'].str.contains("<div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Term'].str.contains("anki")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("</span>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("</div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("anki")].shape[0],0,"HTML tags have been removed")

'OK'

In [727]:
# todo: create function for this
# inspect notes that have spaces in the reading field
# df_notes[df_notes['Term'].str.contains(" ")]

### 12. Check notes for duplicates (shallow check)

In [728]:
assertEquals(has_dupes(df_notes), False, "Duplicates Not Found")

'OK'

### 13. Check for duplicates by term field in notes data frame

In [729]:
def has_dupe_terms(df_in):
    location = df_in['Term'].duplicated()
    return df_in.loc[location].shape[0] != 0

In [730]:
assertEquals(has_dupe_terms(df_notes), False, "Duplicates Found")

'OK'

### 14. Confirm that duplicates dataframe is empty (no dups exist)

In [731]:
dupe = df_notes['Term'].duplicated() #creates list of True/False values
print(df_notes[dupe].shape)
assertEquals(df_notes[dupe].shape[0], 0, "Duplicates dataframe is empty.")

(0, 56)


'OK'

### 15. Inspect an individual card by its term

In [732]:
# Postal service
def inspect_note_by_term(df_in, term):
    return df_in[df_in['Term']==term]

In [733]:
sel1 = inspect_note_by_term(df_notes,'郵便')
sel1

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
3282,1361674609381,1555887839,Japanese Marked abaCheckNuance addDefinition ...,郵便,ゆうびん,"mail, postal service",,,〒,"Common word, Noun",...,,,,,,,,,,


### Save Point, Commit, Bonfire (for you Souls fans)*

At the point in time of the data extraction where the (meta) tag information is made available, we can treat it to both clarify (rename poorly worded tags) & reduce (delete unneeded tags). Since we now have all fields split into their own columns as well, we can treat (modifiy & improve) the columns as well, in a 1-2 process: (1) Fix the tags & (2) Fix the columns
*https://en.wikipedia.org/wiki/Souls_(series)

In [734]:
def shorten_list(takeIn, takeOut):
    temp = takeIn.lower().split() # split all the words into a list
    temp2 = [word for word in temp if word.lower() not in takeOut] # create a shorter list of words minus the take-outs
    return ' '.join(temp2) # return that shorter list as a string

In [735]:
tag_remove_list = ['japanese', 'checkpicture', 'complete', 'haspicture', 'nomemo',
                   'researched', 'aaaeditthis', 'addaudio', 'addaudio2', 'addaudioNow',
                   'addmore','adjustformatting', 'hascomparison', 'hasmnemonic',
                   'customediting','wikidefinition', 'givewill','addaudionow','addprompt',
                   'checknuance','giveyaneury','hastextimage', 'marked', 'addpicture',
                   'addexampletranslation','basicnumeric', 'genkiplus', 'hasaudio',
                   'nativeaudio', 'adddefinition','addexamples', 'addjapaneseprompt',
                   'computervoice','haspoliteprefix','nongoo','customdefinition','hashint',
                   'abahipriorityfix','kaki','mcd','nobodyknows+','missingwordtype',
                   'image','duplicate', 'hasprompt', 'ninshiki','abachecknuance',
                   'hasflag','things', 'jim', 'hasunicode', 'editthis','aaahipriority',
                   'hassimpledef', 'givecodie', 'forjimmy', 'hasnativeaudio', 'givejimmy2',
                   'checkaudio', 'checkwriting', 'hasjlptlevel', 'makekaki', 'checknuance2',
                   'checkagain', 'newaudio', 'mail', 'checkexamples','elementaryschool',
                   'nvc', 'checkprompt', 'gavejimmy', 'addnativeaudio','checkreading',
                   'givecodieapril', 'activated', 'fixformatting','hasplacesuffix',
                   'hassuffix','addtranslation','addnewcardtype','addnuance','addtextimage',
                   'semicomplete', 'removeroboaudio','fixaudio','hasgramconj', 
                   'addkanji','changenotetype', 'famous', 'kuverb',
                   'givwill','karutapoems', 'map', 'hasvisualcomparison','picturekaki',
                   'jyugemu', '2018', 'type1', 'hasslang', 'apologies',
                   'month', 'definitionresearched','soundshift', 'basics1', 'tsuverb',
                   'facebook', 'uverb', 'checkfrequency', 'degree', 'hasdefinition',
                   'addtransliteration', 'dnd', 'introductions', 'adjustprompt',
                   'job', 'particle', 'services', 'mature', 'splitpictures', 
                   'egaki', 'type5k', 'intimate','extrainfo', 'irregular', 'unlisted',
                   'fromwiki', 'checkdifference','addpronunciationdiagram', 'reset',
                   'currentevents', 'doubletextimage', 'comparison', 'verbscompoundpast2',
                   'attention', 'addmemo', 'averb', 'radio','hasascii', 'fontadjusted',
                   'haspronunciation', 'borroweddefinition','alphabet', 'graphics',
                   'chiebukuro', 'duolingo', 'ateji', 'fact','type5s', 'fixpicture',
                   'politebydefault', 'objects','sensitive', 'groupword', 'addmnemonic',
                   'hasmore', 'quote', 'checkformatting','overlap', 'kotobankdef',
                   'hasrudeness', 'changedeck', 'specialformatting','yoga',
                   'hasjapaneseprompt', 'hasprefix','questionword', 'business', 
                   'postoffice', 'firstten', 'money', 'robotvoice2', 'ichidan', 'godan',
                   'weather','count', 'nodefinition', 'muverb', 'addcomparisonchart', 
                   'ruverb', 'phone', 'conjugated','haddiv','vulgar','fromkaruta',
                   'karutamanual', 'teform', '2019'
                  ]

### 16. Remove unneeded tags (meta-data)

In [736]:
# survey a few notes to see example tag data
df_notes.head(3)

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
0,1331799797110,1511481489,Japanese Marked abaCheckNuance checkNuance co...,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",...,,,,,,,,,,
1,1331799797112,1511481489,Japanese complete noMemo researched wwwjdic,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,,,,,,,,,,
2,1331799797113,1511481489,Japanese Marked abaCheckNuance checkNuance co...,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,...,,,,,,,,,,


In [737]:
# likely useful tags: katakana, Waseigo, Food, Phrases, casual, restaurant, travel, commonWord, noun, suruVerb

df_notes_001_less_tags = df_notes.copy() #originally "df_notes_less_tags"
df_notes_001_less_tags['tags'] = df_notes_001_less_tags['tags'].apply(lambda x: shorten_list(str(x), tag_remove_list))

print_before_after(df_notes['tags'].iloc[0], df_notes_001_less_tags['tags'].iloc[0],"Tags for " + df_notes['Term'].iloc[0])

assertEquals("Japanese" in df_notes['tags'].iloc[0].split(), True, "Contains Tag 'Japanese'")
assertEquals("Japanese" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'Japanese'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before:  Japanese Marked abaCheckNuance checkNuance complete noMemo researched wwwjdic yojijukugo 
---------------------------------------------------------------------------
After: wwwjdic yojijukugo
---------------------------------------------------------------------------


'OK'

### 17. Rename useful tags (meta-data) that were poorly named

In [738]:
# replace list (formerly named 'tag_replace_list')
tag_rename_dict = {
    'aalowfrequency':'rare checked', 'aatechnical':'technical checked', 'aaanonkaiwa':'nonconvo checked',
    'wwwjdic':'fromdict', 'expression':'phrase', 'numberonly':'number',
    'grammarpoint':'grammar', 'jisho':'fromdict', 'pointingword':'directions',
    'geometry':'math technical', 'genki':'textbook', 'jpn202':'college',
    'jpn201':'college', 'jpn101':'college', 'jpn102':'college', 'kentei':'fromexam',
    'proficiencytest':'fromexam', 'bodypart':'body', '5kyuu':'fromexam',
    'linguisticreference':'technical', 'conversation':'convo',
    'fromconvo':'convo', 'culturepoint':'culture', 'checkednuance':'checked',
    'checkedpictures':'checked', 'checkednuance':'checked', 'medical':'technical',
    'anatomy':'body', 'places':'place', 'animals':'animal',
    'newspaperterm':'fromnewspaper', 'checkedreading':'checked',
    'abbreviation':'abbr','firstsemester':'semester1','onecharacter':'len1',
    'sentence':'phrase', 'verbs':'verb', 'convook':'checked convo','inuse':'checked',
    'nuancechecked':'checked','insects':'animal insect','sightseeing':'travel',
    'accessories':'clothing', 'grammarsuffix':'suffix', 'oceanlife':'animal ocean',
    'science':'technical', 'written':'nonconvo', 'notrare':'checked',
    'aajoke':'silly', 'intonationcompare':'hassimilar', 'ij':'textbook',
    'goodcard':'inspect','aahilevel':'challenging inspect', 'ijvocab':'textbook',
    'cliothing':'clothing','unused':'nonconvo rare checked',
    'aaunused':'nonconvo rare checked', 'samesound':'hassame','animals':'animal',
    'dictionary':'fromdict','usuallywritteninkana':'kana',
    'abVeryRare':'rare checked', 'yojijukugo':'rare idiom', 'abcasual':'casual checked convo',
    'literaryform':'nonconvo', 'onomatopoeiclike':'onomatopoeic','kenjo':'humble',
    'colors':'color', 'forest':'nature','flower':'plant nature', 'aaok':'checked',
    'questions': 'question', 'adverbs':'adverb','book2':'textbook',
    'book1':'textbook','proficiencytest':'fromtest','animalscomplete':'animal',
    'sonkei':'respectful','eating':'food','fruit':'food','neverused':'nonconvo rare',
    'domainspecific':'technical','seaons':'season','seasons':'season',
    'prefecture':'place','plantpart':'plant', "hakataben":"dialect", "fish":"animal fish",
    "transitive":"transitive verb", "intransitive":"intransitive verb",
    "aaunecessary":"nonconvo checked", "vegetables":"vegetable food plant",
    "counters":"counter", "senmonyougo":"technical", "countries":"country place",
    "date":"datesandtime", "rarelyused":"rare", "aaakaiwa":"convo checked", "cool":"inspect",
    "investigate":"inspect","challenging":"inspect","names":"name",'qanda':'question',
    'hasquestion':'question', "感情のもとにあったニーズ":"phrase rare","phrases":'phrase'
}

#todo: investigate:
#editformatting,  datesandtime, linguistics, reference, adult, adjustpicture, checkpronunciation, addhint, challenging, inspect

In [739]:
def replace_list(takeIn, replaceDict):
    temp = takeIn.lower().split()
    temp2 = []
    for word in temp:
        if word in replaceDict:
            temp2.append(replaceDict.get(word)) # if the word exists in the dictionary, replace it
        else:
            temp2.append(word) # if the word doesnt't exist in the dictionary, leave it alone
    return ' '.join(temp2) # return that shorter list as a string

# inspect further:
# multiwriting, multimeaning, multipicture, multiterm, multireading, mergeterms, checkpronunciation, customterm,
# goodcard, personalized, silly, addjlptlevel, checkpronunciation, mergeterms, customterm, transportation vs travel

# categorize: iadjective, naajective, verb, counter, commonword, suruverb, pronoun, question, phrases, kuverb, godan, ichidan, intransitive, transitive, noun, adverbialnoun

In [740]:
df_notes_002_better_tags = df_notes_001_less_tags.copy() # originally "df_notes_better_tags"
df_notes_002_better_tags['tags'] = df_notes_002_better_tags['tags'].apply(lambda x: replace_list(str(x), tag_rename_dict))

print_before_after(df_notes_001_less_tags['tags'].iloc[0], df_notes_002_better_tags['tags'].iloc[0], "Tags for " + df_notes_002_better_tags['Term'].iloc[0])

assertEquals("wwwjdic" in df_notes_001_less_tags['tags'].iloc[0].split(), True, "Contains Tag 'wwwjdic'")
assertEquals("wwwjdic" in df_notes_002_better_tags['tags'].iloc[0].split(), False, "Contains Tag 'wwwjdic'")
assertEquals("fromdict" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'fromdict'")
assertEquals("fromdict" in df_notes_002_better_tags['tags'].iloc[0].split(), True, "Contains Tag 'fromdict'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before: wwwjdic yojijukugo
---------------------------------------------------------------------------
After: fromdict rare idiom
---------------------------------------------------------------------------


'OK'

In [741]:
# inspect tag strings, notice duplicate occurances of tags
df_notes_002_better_tags['tags'].value_counts()[:20]

                                       1506
fromdict                                778
fromtest textbook                       582
textbook textbook                       431
college textbook textbook               241
verb                                    199
fromdict verb                           143
fromexam                                116
len1                                    107
hiragana college textbook textbook      107
hasrobo                                  99
counter numeric                          96
numeric                                  81
addsimilar hasrobo                       73
college textbook semester1 textbook      70
fromdict media                           68
fromexam textbook                        64
fromdict lyrics                          62
convo                                    57
n3 fromdict transitive verb verb         56
Name: tags, dtype: int64

We can attempt to inspect which tags are most common, in which combinations, and which words would be ideal
for further additional metadata. However, **our tags are still lumped together** at this point. Also, there is
reason to believe that **some tags are showing up multiple times in the same tag string**. In order to properly count tag frequency, the duplicates must be confirmed absent (ie. found & removed). Then, the occurance (word frequency) of each tag may then be summed up for the tags column.

### 18. Inspect a note suspected for tag duplication

In [742]:
def inspect_note_by_id(df_in, nid):
    return df_in[df_in['nid']==nid]

In [743]:
# confirm that a particular note has tag duplicates
# crimison note id: 1369286386384
note_id_1 = 1369286386384
assertEquals(inspect_note_by_id(df_notes_002_better_tags,note_id_1).tags.values[0],
             "fromexam color fromexam len1","Four tags total with two duplicates exist")

'OK'

In [744]:
# example of item with tag duplication
sel2 = inspect_note_by_id(df_notes_002_better_tags,note_id_1)
sel2

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
3845,1369286386384,1511481489,fromexam color fromexam len1,紅,くれない,<div>deep red; crimson</div><div><br /></div><...,,,,"<div>Common word, Noun</div><div>Common word, ...",...,,,,,,,,,,


### 19. Remove duplicate tags (convert tag strings > lists > sets > strings)

In [745]:
# Converts a tag string to a list to a set back to a string (this removes the duplicates)
def remove_dupes(t):
    temp = list(set(t.lower().split()))
    return ' '.join(temp) # return as string

In [746]:
df_notes_003_tags_no_dups = df_notes_002_better_tags.copy()
df_notes_003_tags_no_dups['tags'] = df_notes_003_tags_no_dups['tags'].apply(lambda x: remove_dupes(str(x)))

In [747]:
# determines if an individual tag substring exists in a larger tags list string
def tag_exists(tags, tag):
    return 1 if tag in tags.split() else 0

In [748]:
print(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0])
assertEquals(tag_exists(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0],"len1"), 1, "tag 'len1' remains")
assertEquals(tag_exists(inspect_note_by_id(df_notes_003_tags_no_dups,note_id_1).tags.values[0],"fromexam"), 1, "tag 'fromexam' remains")

fromexam color len1


'OK'

It appears we have most, if not all, of the data we need to start. The format of the dates though is not yet human readable. Let's fix that.

### 20. Convert (& preserve) note ID to note creation date

In [749]:
#dueNum = 782 # this represents days from collection creation date
#crt = 1357635600 # this represents the collection creation date #todo: query dynamically from database
#print("mid 'model id': " + time.ctime(int("1768161991"))) # 1 day = 86400 seconds

df_notes_004_with_date = df_notes_003_tags_no_dups.copy()
df_notes_004_with_date['NoteCreated']= pd.to_datetime(df_notes_004_with_date['nid'],unit='ms')
df_notes_004_with_date['NoteCreated'] = df_notes_004_with_date['NoteCreated'].dt.date
df_notes_004_with_date.head()

print_before_after(df_notes_003_tags_no_dups['nid'].iloc[0], df_notes_004_with_date['NoteCreated'].iloc[0],"Term " + df_notes_004_with_date['Term'].iloc[0])

assertEquals(df_notes_004_with_date['nid'].iloc[0], 1331799797110, "Note ID is in Epoch Units")
assertEquals(str(df_notes_004_with_date['NoteCreated'].iloc[0]), "2012-03-15", "Note ID is in datetime date format year-month-day")

---------------------------------------------------------------------------
Term 臨機応変
---------------------------------------------------------------------------
Before: 1331799797110
---------------------------------------------------------------------------
After: 2012-03-15
---------------------------------------------------------------------------


'OK'

### 21. Generate Note Last Modified Date from "Mod" ID

In [750]:
df_notes_005_last_modified = df_notes_004_with_date.copy()
df_notes_005_last_modified['mod'] = pd.to_datetime(df_notes_005_last_modified['mod'],unit='s')
df_notes_005_last_modified['mod'] = df_notes_005_last_modified['mod'].dt.date

assertEquals(str(df_notes_005_last_modified['mod'].iloc[0]), "2017-11-23", "Note last modified is in datetime date format year-month-day")

'OK'

### 22. Remove rare words, phrases, expressions, questions & sentences from notes

In [751]:
df_notes_006_only_vocab_no_rare = df_notes_005_last_modified.copy()
print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

(8381, 57)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated
0,1331799797110,2017-11-23,fromdict rare idiom,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",...,,,,,,,,,,2012-03-15
1,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,,,,,,,,,,2012-03-15
2,1331799797113,2017-11-23,fromdict,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,...,,,,,,,,,,2012-03-15


In [752]:
sel4 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("rare")]
# https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe
# remove rare words only first
df_notes_006_only_vocab_no_rare = pd.concat([df_notes_006_only_vocab_no_rare, sel4]).drop_duplicates(keep=False)

print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

# todo: assert that no rare words remain in 'df_notes_006_only_vocab_no_rare' by using 'contain("rare")'
# for selection, assert that selection has a row size of 0

(8269, 57)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated
1,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,,,,,,,,,,2012-03-15
2,1331799797113,2017-11-23,fromdict,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,...,,,,,,,,,,2012-03-15
3,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,,,,,,,,,,2012-03-15


In [753]:
# remove phrases, sentences & questions now all at once
sel5 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("phrase")]
sel6 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("sentence")]
sel7 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("question")]
df_notes_006_only_vocab_no_rare = pd.concat([df_notes_006_only_vocab_no_rare, sel5, sel6, sel7]).drop_duplicates(keep=False)

print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

(8048, 57)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated
1,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,,,,,,,,,,2012-03-15
2,1331799797113,2017-11-23,fromdict,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,...,,,,,,,,,,2012-03-15
3,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,,,,,,,,,,2012-03-15


### 23. Create df_notes_midway data frame for progress saving

In [754]:
df_notes_007_midway = df_notes_006_only_vocab_no_rare.copy()

### 24. Export df_notes_midway

In [755]:
df_notes_007_midway.to_csv('datasets/notes_section_2_midway.csv')

### 25. Generate Card Creation Date from Card ID

In [756]:
df_cards_002_created_date = df_cards_001_less_cols.copy()
df_cards_002_created_date['CardCreated'] = pd.to_datetime(df_cards_002_created_date['id'],unit='ms')
df_cards_002_created_date['CardCreated'] = df_cards_002_created_date['CardCreated'].dt.date

assertEquals(str(df_cards_002_created_date['CardCreated'].iloc[0]), "2012-03-15", "Card ID is in datetime date format year-month-day")

'OK'

### 26. Remove cards with no study data associated with them, cards that have been suspended from study

In [757]:
#queue           integer not null,
#      -- -3=sched buried, -2=user buried, -1=suspended,
#      -- 0=new, 1=learning, 2=due (as for type)

df_cards_003_no_new = df_cards_002_created_date.copy()
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=0] # remove cards marked as new
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['reps']!=0] # remove cards that have not been reviewed
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=-1] # remove cards that are currently suspended
# https://stackoverflow.com/questions/18196203/how-to-conditionally-update-dataframe-column-in-pandas
df_cards_003_no_new.loc[df_cards_003_no_new['due'] > 10000, 'due'] = 0 # assign 0 to the due # todo: update w/ last studied date from revlog # todo: comment this line out once you have updated the collection import

print_before_after(df_cards_002_created_date.shape[0], df_cards_003_no_new.shape[0],"Card Rows:")

df_cards_003_no_new.tail(5)

---------------------------------------------------------------------------
Card Rows:
---------------------------------------------------------------------------
Before: 19314
---------------------------------------------------------------------------
After: 8246
---------------------------------------------------------------------------


,id,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated
18898,1549415185907,1549184119039,2,2,2388,87,2410,3,0,2019-02-06
18899,1550403009251,1550402953788,0,-2,2232,1,2410,3,0,2019-02-17
18900,1550403009269,1550402953788,2,2,2317,9,2010,9,2,2019-02-17
18901,1550403084990,1550403040864,0,2,2383,82,2410,3,0,2019-02-17
18902,1550403085003,1550403040864,2,2,2401,93,2410,4,0,2019-02-17


### 27. Confirm that no cards considered 'in learning' are present

In [758]:
sel3 = df_cards_003_no_new[df_cards_003_no_new['due'] == 0]

assertEquals(sel3.shape[0],0,"There are no cards currently in 'learning'.")

'OK'

### 28. Generate Due Date from Due Value

In [759]:
df_cards_004_due_date = df_cards_003_no_new.copy()
df_cards_004_due_date['due'] = pd_crt + df_cards_004_due_date['due'].map(timedelta)
df_cards_004_due_date['due'] = df_cards_004_due_date['due'].dt.date

assertEquals(str(df_cards_004_due_date['due'].iloc[0]), "2015-03-08", "Card due date is in datetime date format year-month-day")

'OK'

### 29. Create df_cards_final data frame for export & further usage

In [760]:
df_cards_final = df_cards_004_due_date.copy()

### 30. Export df_cards_section_2_final

In [761]:
df_cards_final.to_csv('datasets/cards_section_2_final.csv')

### 31. Merge card & note data frames to conduct cross analysis

In [762]:
# now that we have note id's for all the words, we can
# join together these separate dataframes
df_combo = pd.merge(df_notes_007_midway, df_cards_final, on='nid')
print(df_combo.shape)
df_combo.head()

(7957, 66)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,NoteCreated,id,ord,queue,due,ivl,factor,reps,lapses,CardCreated
0,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,2012-03-15,1331799797112,0,2,2015-03-03,149,2080,8,1,2012-03-15
1,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,2012-03-15,1331799797114,0,2,2015-02-04,99,1980,7,0,2012-03-15
2,1331799797117,2017-11-23,fromdict verb,吊るす,つるす,to hang,,,,,...,2012-03-15,1331799797117,0,2,2015-03-17,143,2130,6,1,2012-03-15
3,1331799797118,2017-11-23,fromdict convo checked,和やか,なごやか,"harmonious, peaceful",,,,,...,2012-03-15,1331799797118,0,2,2015-02-06,74,1880,15,3,2012-03-15
4,1331799797121,2017-11-23,fromdict,営業日,えいぎょうび,business day,,,,,...,2012-03-15,1331799797121,0,2,2015-03-03,132,2130,6,1,2012-03-15


The card model has a bunch of columns (fields) with no values in them. These can be taken out for data analysis.

In [763]:
def is_blank (s):
    return not (s and s.strip())

In [764]:
def get_frame_of_cards_by_term(df, t):
    return df.loc[df['Term']==t]

In [765]:
# let's look a a small slice of data, to infer what we may
# we can take a broad overview look at the dataset to more quickly isolate candidates for removal
s = get_frame_of_cards_by_term(df_combo, '発明')
s.head()

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,NoteCreated,id,ord,queue,due,ivl,factor,reps,lapses,CardCreated
2799,1354094556789,2018-12-03,fromtest suruverb commonword noun n3 textbook,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",...,2012-11-28,1354094556789,0,2,2015-06-30,80,1300,73,9,2012-11-28
2800,1354094556789,2018-12-03,fromtest suruverb commonword noun n3 textbook,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",...,2012-11-28,1371807076626,4,2,2021-10-24,1056,1300,20,0,2013-06-21


### 32. Determine which columns (fields) are unused & can be safely removed

In [766]:
col_names = df_combo.columns.values
#print(is_blank(df_combo['Translation2'].iloc[0])) # see that this cell for this row is indeed blank

row_cnt = df_combo.shape[0] # number of rows in df_combo

# https://stackoverflow.com/questions/49677060/pandas-count-empty-strings-in-a-column
empty_strings = pd.DataFrame(df_combo.values == '',columns=col_names) # find all empty strings in a DataFrame
temp_dict = (empty_strings.sum()).to_dict()  # save the location of all empty strings as a DataFrame of booleans
removal_candidates = []
for key in temp_dict.items():
    if key[1] == row_cnt:
        removal_candidates.append(key[0])
print("Removal candidates:", removal_candidates)

Removal candidates: ['Sound3', 'AtoQ', 'AtoQaudio', 'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'blank1', 'blank2']


### 33. Trim unneeded (empty) columns from combo data frame

In [767]:
df_combo_001_less_cols = df_combo.copy()

removal_list = list(removal_candidates + ['queue'])

df_combo_001_less_cols = df_combo_001_less_cols.drop(removal_list,axis=1)

print_before_after(df_combo.shape, df_combo_001_less_cols.shape)
print_before_after(df_combo.columns.values, df_combo_001_less_cols.columns.values)

---------------------------------------------------------------------------
Before: (7957, 66)
---------------------------------------------------------------------------
After: (7957, 56)
---------------------------------------------------------------------------
---------------------------------------------------------------------------
Before: ['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2

It appears that card types are being rendered as numbers, which makes it less human readible. We will fix this. 

### 34. Label card types by their names

In [768]:
# ord stands for 'ordinal' : identifies which of the card templates it corresponds to
print(df_combo_001_less_cols['ord'].value_counts()) # these are the card vectors

# since our dataset contains a single card of a single card vector, & the card vectors
# aren't named/labeled, let's remove the outlier & add the names
df_combo_002_types_labeled = df_combo_001_less_cols.copy()
df_combo_002_types_labeled = df_combo_002_types_labeled.drop(df_combo_002_types_labeled[df_combo_002_types_labeled['ord'] == 11].index)

df_combo_002_types_labeled['ord'].value_counts() # the check shall pass

# now, to map the names onto the card vectors # read:JapaneseReading, recall:EngToJpnTranslate, look:PictureLook, listen:AudioListening
df_combo_002_types_labeled['CardType'] = df_combo_002_types_labeled['ord'].map({0:'read', 2:'recall',4:'look',7:'listen'})
df_combo_002_types_labeled['CardType'].value_counts()

0     6574
4     1100
7      269
2       13
11       1
Name: ord, dtype: int64


read      6574
look      1100
listen     269
recall      13
Name: CardType, dtype: int64

### 35. Create binary exists/not columns based on presence of a given tag

In [769]:
def add_column_by_tag(df, tag):
    df[tag] = df['tags'].apply(lambda x: tag_exists(str(x), tag))

In [770]:
df_combo_003_with_binary = df_combo_002_types_labeled.copy()
inspect_list = ["commonword", "clothing", "animal", "body", "food", "place",
                "textbook", "college", "fromdict", "fromexam",
                "len1", "n1", "n2", "n3", "n4", "n5", 'katakana','hiragana',
                'noun', 'verb', 'convo'
               ]
for item in inspect_list:
    add_column_by_tag(df_combo_003_with_binary, item)

In [771]:
df_combo_003_with_binary.dtypes.value_counts()

object    50
int64     28
dtype: int64

### 36. Create interval quartile sections for visualization purposes

In [772]:
# qcut: Quantile-based discretization function. Discretize variable into equal-sized buckets
# based on rank or based on sample quantiles. For example 1000 values for 10 quantiles would
# produce a Categorical object indicating quantile membership for each data point.
# http://www.datasciencemadesimple.com/quantile-decile-rank-column-pandas-python-2/
df_combo_003_with_binary['ivl_q'] = pd.qcut(df_combo_003_with_binary['ivl'],5,labels=False)
df_combo_003_with_binary.head()

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,...,n2,n3,n4,n5,katakana,hiragana,noun,verb,convo,ivl_q
0,1331799797112,2017-11-23,fromdict,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",...,0,0,0,0,0,0,0,0,0,1
1,1331799797114,2017-11-23,fromdict,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",...,0,0,0,0,0,0,0,0,0,0
2,1331799797117,2017-11-23,fromdict verb,吊るす,つるす,to hang,,,,,...,0,0,0,0,0,0,0,1,0,0
3,1331799797118,2017-11-23,fromdict convo checked,和やか,なごやか,"harmonious, peaceful",,,,,...,0,0,0,0,0,0,0,0,1,0
4,1331799797121,2017-11-23,fromdict,営業日,えいぎょうび,business day,,,,,...,0,0,0,0,0,0,0,0,0,0


Let's further refine the dataframe entries to represent which notes have (1) visual data, (2) audio data, and (3) a L1 ("first language", English in this case) translation. We can represent these with binary values (0 for doesn't exist, 1 for exists).

### 37. Create boolean columns for predictive models

In [773]:
# Laura calls this process "Data Enriching"
intify_list = ['hasPOS','hasVisual','hasAudio','hasMultiMeaning','hasMultiReading','hasSimilar','hasHomophone','hasAltForm']

In [774]:
# https://stackoverflow.com/questions/17383094/how-can-i-map-true-false-to-1-0-in-a-pandas-dataframe
df_combo_003_with_binary['hasPOS'] = df_combo_003_with_binary['PartOfSpeech']!="" #todo: expand upon this, by tagify
df_combo_003_with_binary['hasVisual'] = df_combo_003_with_binary['Picture1']!=""
df_combo_003_with_binary['hasAudio'] = df_combo_003_with_binary['Sound']!=""
df_combo_003_with_binary['hasMultiMeaning'] = df_combo_003_with_binary['Translation2' and 'Translation3' and 'Meaning2']!=""
df_combo_003_with_binary['hasMultiReading'] = df_combo_003_with_binary['Yomi2']!="" # todo: inspect & incorporate venn diagram: https://commons.wikimedia.org/wiki/File:Homograph_homophone_venn_diagram.png
df_combo_003_with_binary['hasSimilar'] = df_combo_003_with_binary['SimilarWords']!=""
df_combo_003_with_binary['hasHomophone'] = df_combo_003_with_binary['SameSoundWords']!="" # write function, detect homophones
df_combo_003_with_binary['hasAltForm'] = df_combo_003_with_binary['Term2' and 'AlternateForms']!= ""

### 38. Drop non-numerical columns from combo data frame

In [775]:
df_combo_004_less_cols = df_combo_003_with_binary.copy()
df_combo_004_less_cols = df_combo_004_less_cols.drop(['Examples','ExamplesAudio',
                            'Meaning1','RelatedWords','Breakdown1','Comparison',
                           'Usage','Prompt1','Prompt2','KakuMCD','IuMCD','ExtraMemo',
                           'Breakdown2','Picture2','Picture3','Picture4','Mnemonic',
                            'Yomi3','gChap','gBook','semester','gNumber','ArabicNumeral',
                            'CounterKanji','SoloLookCards','HinshiMarker','Hint',
                            'mod','Transliteration'],axis=1)
# todo: explore 'mod' (last modified date) as freshness metric

In [776]:
# casts columns of type object to type int as directed, use with caution
def intify_bools(df, col):
    df[col] = df[col].astype(int)

### 39. Ensure numerical/boolean types are encoded properly

In [777]:
df_combo_004_less_cols.dtypes.value_counts()

int64     29
object    21
bool       8
dtype: int64

In [778]:
for item in intify_list:
    intify_bools(df_combo_004_less_cols,item)

In [779]:
df_combo_004_less_cols.dtypes.value_counts()

int64     37
object    21
dtype: int64

### 40. Further reduce columns no longer needed

In [780]:
df_combo_004_less_cols = df_combo_004_less_cols.drop(['Picture1','Sound','Sound2',
                              'TagOverflow','Translation2', 'Meaning2','Yomi2','Term2',
                              'SameSoundWords','hasPOS','SimilarWords','AlternateForms',
                            'Translation3'],axis=1)
df_combo_004_less_cols.head(35)[30:]

#selection2 = df_binary.loc[df_binary['hasMultiMeaning']==1]
#selection2.head()

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,verb,convo,ivl_q,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm
30,1342506824725,fromdict,事情,じじょう,circumstances; consideration; conditions; situ...,"Common word, Noun",2012-07-17,1342506824725,0,2015-07-10,...,0,0,2,0,0,0,0,0,0,0
31,1342506824726,fromdict,事柄,ことがら,matter; thing; affair; circumstance,"Common word, Noun",2012-07-17,1342506824726,0,2015-11-10,...,0,0,2,0,0,0,0,1,0,0
32,1342506824727,fromdict,物事,ものごと,things; everything,"Common word, Noun",2012-07-17,1342506824727,0,2016-03-17,...,0,0,2,0,0,0,0,1,0,0
33,1342506824728,fromtest fromdict textbook,方法,ほうほう,"<span style="" font-style: normal; font-weight:...",,2012-07-17,1342506824728,0,2015-10-24,...,0,0,3,0,0,0,0,0,0,0
34,1342506824729,fromdict,行為,こうい,"act, deed, conduct","Common word, Noun",2012-07-17,1342506824729,0,2015-04-16,...,0,0,0,0,0,0,0,1,0,0


### 41. Count syllable count & character length for each term

In [781]:
df_combo_005_with_len = df_combo_004_less_cols.copy()

df_combo_005_with_len['TermLen'] = df_combo_005_with_len['Term'].str.len()
df_combo_005_with_len['Syllables'] = df_combo_005_with_len['Yomi1'].str.len()
df_combo_005_with_len.loc[df_combo_005_with_len['Syllables'] == 0, 'Syllables'] = df_combo_005_with_len['TermLen']

bins = [0,1,2,4,8,128]
labels = ["[1]","[2]","[3:4]","[5:8]","[9: ]"]
# https://stackoverflow.com/questions/45273731/binning-column-with-python-pandas
df_combo_005_with_len['TermLenGroup'] = pd.cut(df_combo_005_with_len['TermLen'], bins=bins, labels=labels)

#df.loc[df['Grades'] <= 77, 'Grades'] = 100
# https://stackoverflow.com/questions/42815768/pandas-adding-column-with-the-length-of-other-column-as-value
#df_binary2.head(35)[30:]
df_combo_005_with_len.tail(20)[:5]

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,TermLen,Syllables,TermLenGroup
7937,1483483651002,clothing college gairaigo katakana textbook,ネクタイ,,necktie,,2017-01-03,1485707813168,4,2020-03-29,...,1,1,0,0,0,0,0,4,4,[3:4]
7938,1483483651027,clothing college gairaigo katakana textbook,ジャケット,,jacket,,2017-01-03,1485708017322,4,2020-09-27,...,1,1,0,0,0,0,0,5,5,[5:8]
7939,1485703825555,clothing gairaigo katakana n5 commonword noun ...,ズボン,,1. trousers; pants,Noun,2017-01-29,1485708950599,4,2020-06-11,...,1,1,0,0,0,0,0,3,3,[3:4]
7940,1485705402623,clothing gairaigo katakana commonword noun,ブラジャー,,1. bra; brassiere,,2017-01-29,1485707774754,4,2019-05-02,...,1,0,0,0,0,0,0,5,5,[5:8]
7941,1489373157595,,細切り,ほそぎり,thin strips; matchstick-like strips; julienned...,Noun,2017-03-13,1489373228966,0,2017-03-25,...,0,0,0,0,0,0,0,3,4,[3:4]


### 42. Inspect the longest syllable entries

In [782]:
df_many_syl = df_combo_005_with_len.copy()
many_syl = df_many_syl['Syllables'] > 15
df_many_syl.loc[many_syl] #todo: check nid of 1391477462767

,nid,tags,Term,Yomi1,Translation,PartOfSpeech,NoteCreated,id,ord,due,...,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,TermLen,Syllables,TermLenGroup
234,1346057958628,inspect fromdict fromnewspaper history culture,東京電力福島・第１原発事故,とうきょうでんりょくふくしま・だいいちげんぱつじこ,,,2012-08-27,1346057958628,0,2015-07-07,...,0,0,0,0,0,0,0,13,25,[9: ]
301,1346215143756,fromdict datesandtime numeric,1837～1901年,せんはっぴゃくさんじゅうななねんからせんきゅうひゃくいちねん,,,2012-08-29,1346215143756,0,2015-07-14,...,0,0,0,0,0,0,0,10,30,[9: ]
414,1346216471844,counter datesandtime fromdict numeric,千九百八十九年,せんきゅうひゃくはちじゅうきゅうねん,,,2012-08-29,1346216471844,0,2015-11-24,...,0,0,0,0,0,0,0,7,18,[5:8]
434,1346220179889,counter datesandtime fromdict numeric,千九百四十五年,せんきゅうひゃくよんじゅうごねん,,,2012-08-29,1346220179889,0,2016-01-20,...,0,0,0,0,0,0,0,7,16,[5:8]
439,1346220179894,fromdict fromnewspaper,国連安全保障理事会,こくれんあんぜんほしょうりじかい,United Nations Security Board of Directors,,2012-08-29,1346220179894,0,2015-05-08,...,1,1,0,0,0,0,0,9,16,[9: ]
440,1346220179894,fromdict fromnewspaper,国連安全保障理事会,こくれんあんぜんほしょうりじかい,United Nations Security Board of Directors,,2012-08-29,1379660811751,4,2018-05-21,...,1,1,0,0,0,0,0,9,16,[9: ]
3048,1354972584071,fromdict katakana technical,オブジェクト指向プログラミング,オブジェクトしこうプログラミング,object-oriented programming,"Noun, Computer terminology",2012-12-08,1354972584071,0,2015-07-09,...,0,0,0,0,0,0,0,15,16,[9: ]
5743,1387411183585,numeric datesandtime,千九百八十七年,せんきゅうひゃくはちじゅうななねん,,,2013-12-18,1387411238321,0,2015-11-26,...,0,0,0,0,0,0,0,7,17,[5:8]
7535,1411020895026,math technical,二次方程式の解の公式,にじほうていしきのかいのこうしき,the quadratic formula,"Noun, technical, math",2014-09-18,1411021140081,0,2015-08-07,...,1,0,0,0,0,0,0,10,16,[9: ]


In [783]:
df_combo_005_with_len.columns.values

array(['nid', 'tags', 'Term', 'Yomi1', 'Translation', 'PartOfSpeech',
       'NoteCreated', 'id', 'ord', 'due', 'ivl', 'factor', 'reps',
       'lapses', 'CardCreated', 'CardType', 'commonword', 'clothing',
       'animal', 'body', 'food', 'place', 'textbook', 'college',
       'fromdict', 'fromexam', 'len1', 'n1', 'n2', 'n3', 'n4', 'n5',
       'katakana', 'hiragana', 'noun', 'verb', 'convo', 'ivl_q',
       'hasVisual', 'hasAudio', 'hasMultiMeaning', 'hasMultiReading',
       'hasSimilar', 'hasHomophone', 'hasAltForm', 'TermLen', 'Syllables',
       'TermLenGroup'], dtype=object)

### 43. Remove unneeded Translation & PartOfSpeech columns

In [784]:
df_combo_006_less_cols = df_combo_005_with_len.copy()
df_combo_006_less_cols = df_combo_006_less_cols.drop(['Translation','PartOfSpeech'],axis=1)

In [785]:
# labels terms by their jlpt level.
# bear in mind that some terms have multiple jlpt levels.
# this function merely assigns the lowest associated jlpt level with a term. 
def label_jlpt_lvl (row):
    if row['n5'] == 1 :
        return 5
    elif row['n4'] == 1:
        return 4
    elif row['n3'] == 1:
        return 3
    elif row['n2'] == 1:
        return 2
    elif row['n1'] == 1:
        return 1
    else:
        return None

### 44. Assign JLPT number to words with JLPT "N" levels

In [786]:
df_combo_007_jptl_lvl = df_combo_006_less_cols.copy()
df_combo_007_jptl_lvl['jlpt_lvl'] = df_combo_007_jptl_lvl.apply (lambda row: label_jlpt_lvl(row), axis=1)

In [787]:
df_combo_007_jptl_lvl.columns.values

array(['nid', 'tags', 'Term', 'Yomi1', 'NoteCreated', 'id', 'ord', 'due',
       'ivl', 'factor', 'reps', 'lapses', 'CardCreated', 'CardType',
       'commonword', 'clothing', 'animal', 'body', 'food', 'place',
       'textbook', 'college', 'fromdict', 'fromexam', 'len1', 'n1', 'n2',
       'n3', 'n4', 'n5', 'katakana', 'hiragana', 'noun', 'verb', 'convo',
       'ivl_q', 'hasVisual', 'hasAudio', 'hasMultiMeaning',
       'hasMultiReading', 'hasSimilar', 'hasHomophone', 'hasAltForm',
       'TermLen', 'Syllables', 'TermLenGroup', 'jlpt_lvl'], dtype=object)

In [788]:
df_combo_007_jptl_lvl['jlpt_lvl'].value_counts()

3.0    176
5.0    147
4.0    100
2.0     31
1.0     22
Name: jlpt_lvl, dtype: int64

### 45. Create dummy variables for card type

In [789]:
df_combo_008_dummies = df_combo_007_jptl_lvl.copy()

df_combo_008_dummies = pd.get_dummies(df_combo_008_dummies, columns=['CardType'])

In [790]:
df_combo_008_dummies.tail(10)[:5]

,nid,tags,Term,Yomi1,NoteCreated,id,ord,due,ivl,factor,...,hasHomophone,hasAltForm,TermLen,Syllables,TermLenGroup,jlpt_lvl,CardType_listen,CardType_look,CardType_read,CardType_recall
7947,1523892839900,commonword noun n5,万年筆,まんねんひつ,2018-04-16,1523893083493,0,2018-06-14,27,2210,...,0,0,3,6,[3:4],5.0,0,0,1,0
7948,1523892839900,commonword noun n5,万年筆,まんねんひつ,2018-04-16,1523893083509,7,2020-10-01,515,2410,...,0,0,3,6,[3:4],5.0,1,0,0,0
7949,1523892839900,commonword noun n5,万年筆,まんねんひつ,2018-04-16,1523893129423,2,2018-06-14,13,2210,...,0,0,3,6,[3:4],5.0,0,0,0,1
7950,1523892839900,commonword noun n5,万年筆,まんねんひつ,2018-04-16,1524841320859,4,2019-05-05,4,2050,...,0,0,3,6,[3:4],5.0,0,1,0,0
7951,1549184119039,,閏年,うるうどし,2019-02-03,1549184129288,0,2019-08-21,108,2410,...,0,0,2,5,[2],NaN,0,0,1,0


### 46. Group notes by ID to determine card type overlap, simple totals per note

In [791]:
# https://stackoverflow.com/questions/23919563/merge-rows-of-a-dataframe-in-pandas-based-on-a-column
df_combo_009_grouped_notes = df_combo_008_dummies.groupby(['nid']).sum()

In [792]:
# this data frame will provide total reps per term, total lapses per term, and vectors (card types) per term 
df_combo_009_grouped_notes.tail(20)[-5:]

df_notes_008_totals = df_combo_009_grouped_notes.copy()
df_notes_008_totals = df_notes_008_totals.drop(['id','ivl','ord','factor','commonword','clothing','animal',
    'body','food','place','textbook','college','fromdict','fromexam','len1','n1','n2','n3','n4','n5',
    'ivl_q','hasVisual','hasAudio','hasMultiMeaning','hasMultiReading','hasSimilar','hasHomophone',
    'hasAltForm','TermLen','Syllables','jlpt_lvl'],axis=1)
df_notes_008_totals.tail(20)[-5:]

,reps,lapses,katakana,hiragana,noun,verb,convo,CardType_listen,CardType_look,CardType_read,CardType_recall
nid,,,,,,,,,,,
1508012628565,10,1,0,0,0,0,0,1,1,1,1
1523892839900,31,4,0,0,4,0,0,1,1,1,1
1549184119039,8,0,0,0,0,0,0,0,0,1,1
1550402953788,12,2,0,0,2,0,2,0,0,1,1
1550403040864,7,0,0,0,0,0,0,0,0,1,1


In [793]:
df_notes_008_totals = df_notes_008_totals.rename(columns={'reps':'reps_total','lapses':'lapses_total',
    'CardType_listen':'hasListenCard', 'CardType_recall':'hasTranslateCard',
    'CardType_read':'hasReadCard', 'CardType_look':'hasPictureCard'})

In [794]:
df_notes_008_totals.tail(20)[-5:]

,reps_total,lapses_total,katakana,hiragana,noun,verb,convo,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
nid,,,,,,,,,,,
1508012628565,10,1,0,0,0,0,0,1,1,1,1
1523892839900,31,4,0,0,4,0,0,1,1,1,1
1549184119039,8,0,0,0,0,0,0,0,0,1,1
1550402953788,12,2,0,0,2,0,2,0,0,1,1
1550403040864,7,0,0,0,0,0,0,0,0,1,1


### 47. Group notes by ID to find simple average means per note

In [795]:
df_notes_009_means = df_combo_008_dummies.copy()
df_notes_009_means = df_notes_009_means.groupby(['nid']).mean()

In [796]:
df_notes_009_means = df_notes_009_means.drop(['id','ord','commonword','clothing','animal','body',
    'food','place','textbook','college','fromdict','fromexam','len1','n1','n2','n3','n4','n5','ivl_q',
    'hasVisual','hasAudio','hasMultiMeaning','hasMultiReading','hasSimilar','hasHomophone','hasAltForm',
    'TermLen','Syllables','jlpt_lvl','CardType_listen','CardType_recall',
    'CardType_read','CardType_look'],axis=1)
df_notes_009_means = df_notes_009_means.rename(columns={'ivl':'mean_ivl','factor':'mean_factor',
                                                        'reps':'mean_reps','lapses':'mean_lapses'})
df_notes_009_means.tail(20)[-5:]

,mean_ivl,mean_factor,mean_reps,mean_lapses,katakana,hiragana,noun,verb,convo
nid,,,,,,,,,
1508012628565,130.50,2360.0,2.50,0.25,0.0,0.0,0.0,0.0,0.0
1523892839900,139.75,2220.0,7.75,1.00,0.0,0.0,1.0,0.0,0.0
1549184119039,97.50,2410.0,4.00,0.00,0.0,0.0,0.0,0.0,0.0
1550402953788,5.00,2210.0,6.00,1.00,0.0,0.0,1.0,0.0,1.0
1550403040864,87.50,2410.0,3.50,0.00,0.0,0.0,0.0,0.0,0.0


### 48. Combine note totals, note means & general notes

In [797]:
df_notes_010_combo = df_notes_009_means.copy()
df_notes_010_combo = pd.merge(df_notes_010_combo, df_notes_008_totals,on='nid')
df_notes_010_combo.head(5)

,mean_ivl,mean_factor,mean_reps,mean_lapses,katakana_x,hiragana_x,noun_x,verb_x,convo_x,reps_total,lapses_total,katakana_y,hiragana_y,noun_y,verb_y,convo_y,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
nid,,,,,,,,,,,,,,,,,,,,
1331799797112,149.0,2080.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,8,1,0,0,0,0,0,0,0,1,0
1331799797114,99.0,1980.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,7,0,0,0,0,0,0,0,0,1,0
1331799797117,143.0,2130.0,6.0,1.0,0.0,0.0,0.0,1.0,0.0,6,1,0,0,0,1,0,0,0,1,0
1331799797118,74.0,1880.0,15.0,3.0,0.0,0.0,0.0,0.0,1.0,15,3,0,0,0,0,1,0,0,1,0
1331799797121,132.0,2130.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,6,1,0,0,0,0,0,0,0,1,0


In [798]:
# Import in Review Log data

In [799]:
df_revlog = pd.read_sql_query("SELECT * FROM revlog", cnx)

In [800]:
print(df_revlog.shape)
df_revlog.head()

(114129, 9)


,id,cid,usn,ease,ivl,lastIvl,factor,time,type
0,1332393018515,1331799797110,0,1,0,0,2500,6673,0
1,1333279992123,1331799797110,0,4,8,0,2600,11656,0
2,1333280001016,1331799797112,0,4,8,0,2600,8887,0
3,1333280097922,1331799797113,0,1,0,0,2500,29162,0
4,1333280107916,1331799797114,0,4,8,0,2600,9987,0


In [801]:
def inspect_card_by_id(df_in, id_num, id_name):
    return df_in[df_in[id_name]==id_num]

In [802]:
df_revlog_001_review_date = df_revlog.copy()
df_revlog_001_review_date = df_revlog_001_review_date.rename(columns={'id':'rid'})
df_revlog_001_review_date['ReviewDate']= pd.to_datetime(df_revlog_001_review_date['rid'],unit='ms')
df_revlog_001_review_date['ReviewDate'] = df_revlog_001_review_date['ReviewDate'].dt.date
df_revlog_001_review_date.head()

assertEquals(df_revlog_001_review_date['rid'].iloc[0], 1332393018515, "Note ID is in Epoch Units")
assertEquals(str(df_revlog_001_review_date['ReviewDate'].iloc[0]), "2012-03-22", "Note ID is in datetime date format year-month-day")

'OK'

In [804]:
current_note_id = inspect_card_by_id(df_cards_final, df_revlog['cid'].iloc[0], 'id')['nid'].iloc[0]

print(current_note_id)

#print("Note ID: ", inspect_note_by_id(df_notes_final_001,current_note_id)['nid'].iloc[0])
#print("Term: ", inspect_note_by_id(df_notes_final_001,current_note_id)['Term'].iloc[0])
#print("Translation: ", inspect_note_by_id(df_notes_final_001,current_note_id)['Translation'].iloc[0])

inspect_card_by_id(df_cards_final, df_revlog['cid'].iloc[0], 'id') # in df_cards_final (X) look for a card of 'card id' X in the column name 'id' Z

1331799797110


,id,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated
0,1331799797110,1331799797110,0,2,2015-03-08,65,1680,10,1,2012-03-15


In [805]:
inspect_card_by_id(df_revlog_001_review_date, df_revlog['cid'].iloc[0], 'cid')

,rid,cid,usn,ease,ivl,lastIvl,factor,time,type,ReviewDate
0,1332393018515,1331799797110,0,1,0,0,2500,6673,0,2012-03-22
1,1333279992123,1331799797110,0,4,8,0,2600,11656,0,2012-04-01
80368,1397571358201,1331799797110,4480,1,-60,-60,2500,4292,0,2014-04-15
80369,1397571360841,1331799797110,4480,2,-600,-60,2500,2636,0,2014-04-15
80370,1397571363081,1331799797110,4480,2,1,-600,2280,2238,0,2014-04-15
80377,1397622541113,1331799797110,4490,3,2,1,2280,4023,1,2014-04-16
83544,1400914850867,1331799797110,4958,2,12,2,2130,3323,1,2014-05-24
93052,1410177777778,1331799797110,6257,2,44,12,1980,2300,1,2014-09-08
98902,1414062295845,1331799797110,6748,2,51,44,1830,16176,1,2014-10-23
104064,1420285596480,1331799797110,7154,2,65,51,1680,11880,1,2015-01-03
